In [1]:
import cdsapi
from datetime import datetime, timedelta
from calendar import monthrange
import xarray as xr

#url: https://cds.climate.copernicus.eu/api/v2
#key: 159276:b225e1d7-1242-4152-8576-ce50852fbd4e

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
            'soil_temperature_level_1', 'total_precipitation',
        ],
        'year': [
            '2019',
        ],
        'month': [
            '04','05','06',
            '07', '08', '09',
            '10', '11',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31', 
        ],
        'time': [
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00',
            #13
        ],
        'area': [
            
            38.1445082027146, -97.72654627101196, 38.142173460759004,
            -97.72096882266754,
        ],
        'format': 'netcdf',
    },
    'download.nc')

2022-10-26 16:19:26,942 INFO Welcome to the CDS
2022-10-26 16:19:26,943 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-10-26 16:19:27,162 INFO Request is completed
2022-10-26 16:19:27,164 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data2/adaptor.mars.internal-1666817498.425104-20916-13-a7a8205e-3265-480f-87d3-1fe0de4d6e46.nc to download.nc (26.1K)
2022-10-26 16:19:28,411 INFO Download rate 21K/s    


Result(content_length=26748,content_type=application/x-netcdf,location=https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data2/adaptor.mars.internal-1666817498.425104-20916-13-a7a8205e-3265-480f-87d3-1fe0de4d6e46.nc)

In [158]:

#Abrimos la informacion
data_tp=xr.open_dataset('download.nc')

In [161]:

from ast import Dict


def data_extrator_temp(
    data_tp,
) -> Dict:
    year_2019 = {}
    year = 2019
    # Temperature.
    temperature = data_tp['stl1'].values.ravel()
    # Precipitation.
    precipitation = data_tp['tp'].values.ravel()

    # Ordered month.
    months = list(set([x.to_pydatetime().month for x  in data_tp['time'].to_series()]))

    #Iteration to aggregate the corresponding values per month(day and temp values are added),
    for month in sorted(months):
        #number of days in a month,
        month_range = monthrange(year, month)[1]

        #Generate days in the month.
        days = [x + 1 for x in range(month_range)]

        # number of temperature and precipitation data per day ().
        n_data= len(set([x.to_pydatetime().hour for x  in data_tp['time'].to_series()]))

        month_temp = {}

        # Get the temp
        for day in days:

            #gives the value for temperature and precipitation per hour.
            values_per_hour = {'temperature' : temperature[0:n_data], 'precipitation' : precipitation[0:n_temperature]}

            #Take the corresponding values per day for temperature and precipitation.
            temperature = temperature[n_data:]
            precipitation = precipitation[n_data:]

            #Updates the dictionary and adds the previously calculated values.
            month_temp.update(
            {day : values_per_hour}
            )
        #adds the information of months, days and their temperature and precipitation data  to the main dictionary.
        year_2019.update({
            month : month_temp
        })
    return year_2019

In [163]:
ndvi_ts = data_extrator_temp(data_tp=data_tp)
ndvi_ts

{4: {1: {'temperature': array([276.7997 , 277.23633, 276.80035, 275.9194 , 275.20642, 275.1801 ,
          275.21924, 275.49557, 276.3836 , 278.6238 , 280.27097, 281.5764 ,
          282.44708], dtype=float32),
   'precipitation': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)},
  2: {'temperature': array([279.30023, 279.38553, 279.26434, 278.61548, 277.63577, 277.54602,
          277.49472, 277.20877, 278.18527, 280.28635, 281.2872 , 282.82983,
          284.05   ], dtype=float32),
   'precipitation': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)},
  3: {'temperature': array([281.8713 , 281.61484, 281.27374, 280.63385, 280.3665 , 280.74222,
          281.22693, 281.4071 , 281.9162 , 284.0288 , 284.6892 , 285.7452 ,
          286.45114], dtype=float32),
   'precipitation': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
          4.1128322e-0